This notebook is part of 4 approaches. This notebook implements the Entity Extarction by exploiting the coordiantes of token and using the 'Attention' model. The final assessment is done by comparing the each token labels and if it 'exactly' matches the actual label then only the prediction is considered correct ( this was done to benchmark it with Google AUTO ML part ). 

This model is trained on all labels in one go, therefore like previous approaches it has one model only which predicts all the 4 labels.

In [ ]:
! pip install tensorflow_addons

In [ ]:
import pandas as pd
import spacy
import numpy as np
import json
import tensorflow as tf
import operator
from tensorflow.keras import backend as be
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Input
from tensorflow.keras.models import Model
from keras.constraints import maxnorm
from tensorflow_addons.optimizers import RectifiedAdam
import tensorflow_addons as tfa
from tensorflow.keras.losses import categorical_crossentropy, sparse_categorical_crossentropy, binary_crossentropy


from tensorflow.keras.optimizers import Adam,Adadelta,RMSprop
from tensorflow.keras  import losses
from tensorflow.keras import backend as be
from keras.utils.vis_utils import plot_model
from numpy.random import seed
seed(7)
tf.compat.v1.set_random_seed(7)
import pickle

In [ ]:
# Lets mount the Google Drive and acccess the data
from google.colab import drive
drive.mount('/content/drive')

The code for reading and extracting the information from jsonl file.

##### EXTRACTION OF LABELS START OFFSET AND END OFFSET VALUES and TOKENS

In [ ]:
# as the the file type is jsonl, each invoice is represented by hierarchial json object, therefore
# we need to fetch each invoice in hierarchial manner
# further this will access the folder from where to read the fields, so please change the path 
# as per your specifications.

documents = []

# each invoice belogs to either train, validation, test set
category = []

# with the json object of each invoice we also have another json object with key (label), value (string) pair
# each annotation(label) has been given one initial 'null' value as there cannot be empty dictionary with keys.
placeholder = {'Date':['null'], 'Address': ['null'], 'Name':['null'], 'Amount':['null']}

# this is similar to placeholder but will read the tokens provided by OCR for each invoice.
tokens = {'sentTokens':['null'], 'sentToken_startOffset':['null'], 'sentToken_endOffset':['null'],'boundingPoly':['null']}


# this reads the csvfile which gives list of all json files
list_ = pd.read_csv('/content/drive/MyDrive/......csv',usecols=[0,2], names=['Type', 'Filename'], header=None)
for type_, file_name in list_.itertuples(index=False):

    # read the jsonfile
    with open('/content/drive/MyDrive/..../{}'.format(file_name), 'r', encoding="utf8") as json_file:
        json_list = list(json_file)

    # each invoice is encapsulated in one dictionary with many key-value pairs
    for json_str in json_list:
        result = json.loads(json_str)


        #-----------------READING THE RAW TEXT AND INVOICE TYPE-------------------------------------------
        # extracting the raw text provided by OCR 
        documents.append(result['document']['documentText']['content'])

        # extracting the invoice type (train/validation/test) 
        category.append(type_)

        #------------------EXTRACTING OFFSET VALUES FOR LABELS------------------------------------------
        # As not all invoices has all the 4 annotations we need to keep track of same
        truth_table = {'Date': False, 'Address': False, 'Name': False, 'Amount':False}

        # extracting the annotations and appending them to the placeholder
        for i in range(len(result['annotations'])):
                placeholder[result['annotations'][i]['displayName']].append(result['annotations'][i]['textExtraction']['textSegment'])
                truth_table[result['annotations'][i]['displayName']] = True
        
        # if any annotation not available for invoice them append empty value for same 
        for key, value in truth_table.items():
            if value == False:
                placeholder[key].append({})

        #--------------------------------EXTRACTING TOKENS GIVEN BY OCR for EACH INVOICE and its bounding Polygon--------------------------
        temp = {'tokensOfinvoice':[],'polygons':[]}
        start_off = []
        end_off = []

        # extracting the raw text provided by OCR 
        for i in range(len(result['document']['layout'])):
            temp['tokensOfinvoice'].append(result['document']['layout'][i]['textSegment']['content'])
            temp['polygons'].append(result['document']['layout'][i]['boundingPoly']['normalizedVertices'])
            if 'startOffset' in result['document']['layout'][i]['textSegment']:
                start_off.append(int(result['document']['layout'][i]['textSegment']['startOffset']))
            else:
                start_off.append(0)
            
            if 'endOffset' in result['document']['layout'][i]['textSegment']:
                end_off.append(int(result['document']['layout'][i]['textSegment']['endOffset']))
            else: 
                end_off.append(-1)

        tokens['sentTokens'].append(temp['tokensOfinvoice'])
        tokens['boundingPoly'].append(temp['polygons'])
        tokens['sentToken_startOffset'].append(start_off)
        tokens['sentToken_endOffset'].append(end_off)


# getting the list of all annotations offset and tokens ( removing first null entry )
placeholder['Date'] = placeholder['Date'][1:]
placeholder['Address']= placeholder['Address'][1:]
placeholder['Name'] = placeholder['Name'][1:]
placeholder['Amount'] = placeholder['Amount'][1:]

placeholder['sentTokens'] = tokens['sentTokens'][1:]
placeholder['sentToken_startOffset'] = tokens['sentToken_startOffset'][1:]
placeholder['sentToken_endOffset'] = tokens['sentToken_endOffset'][1:]
placeholder['boundingPoly'] = tokens['boundingPoly'][1:]
# appending the raw text and type of invoice
placeholder['text'] = documents
placeholder['dataType'] = category


# checking for count of all annotations and documents
print(placeholder.keys())
print(len(placeholder['Amount']))
print(len(placeholder['Date']))
print(len(placeholder['Address']))
print(len(placeholder['Name']))
print(len(placeholder['text']))
print(len(placeholder['dataType']))

data = pd.DataFrame.from_dict(placeholder)

# Remember we have dictionary containing the start and end offset, we can extract them 
# separately too, if required.
#--------------------------------------------------------------------------
for keys in ['Date', 'Address', 'Name', 'Amount']:
    # sometimes startoffset might not be present if annotation start from first index of raw text
    data['{}_start'.format(keys)] = data[keys].apply(lambda x: int(x['startOffset']) if 'startOffset' in x.keys()
                        else ( 0 if 'endOffset' in x.keys() else None ))

    # similarly as above endoffset might not be present if the annotations end is last index of raw text                 
for keys in ['Date', 'Address', 'Name', 'Amount']:   
    data['{}_end'.format(keys)] = data[keys].apply(lambda x: int(x['endOffset']) if 'endOffset' in x.keys()
                        else ( -1 if 'startOffset' in x.keys() else None))

data = data.drop(['Date', 'Address', 'Name', 'Amount'], axis = 1)

Lets contruct the target for the models, the OCR gives the startoffset and endoffset for each label, however this is at character. To make the encoding for each label at otken level, we will use simple logic if the startOffset of token lies within the bound of actual label then its part of label else not. This approach has limitation, because if the actual label is sub-part of token then this will exclude that token.This is one limitation of using the output of OCR. This same cause is also the reason for the AutoML bad performance, because any model will work on token level, however if the labelling is done by taking the sub-span within token then model is not able to take that into factor.

In [ ]:

# placeholder to keep the target of each label
import math
placeholder = {'Date':['null'], 'Address': ['null'], 'Name':['null'], 'Amount':['null']}
for i,j in placeholder.items():
    for a,b,c,d in zip(data['sentToken_startOffset'],data['sentToken_endOffset'],data['{}_start'.format(i)], data['{}_end'.format(i)]):

        if not math.isnan(c):

            start_left = np.array(a) >=c
            start_right = np.array(a) <=d
            start_truth_table = start_left == start_right
            
            end_left = np.array(b)  >= c
            end_right = np.array(b) <=d
            end_truth_table = end_left == end_right
            truth_table = start_truth_table + end_truth_table 

            placeholder['{}'.format(i)].append(truth_table.astype(int))
        else:
            placeholder['{}'.format(i)].append(list(np.zeros(len(a)).astype(int)))

# removing the first null entry.
placeholder['dateTarget'] = placeholder['Date'][1:]
placeholder['addressTarget']= placeholder['Address'][1:]
placeholder['nameTarget'] = placeholder['Name'][1:]
placeholder['amountTarget'] = placeholder['Amount'][1:]

del placeholder['Address'], placeholder['Amount'], placeholder['Date'], placeholder['Name']

# creating final dataframe 
y = pd.DataFrame.from_dict(placeholder)
data = pd.concat([data,y], axis =1)

In [ ]:
# If any token is present in label, then the the max value in the list for that sentence for that label will be 1, 
# hence if eveything is fine then we shoud see same numbers for all labels. However as highlighted due to slicing of token in case of amount, 
# it is have mis match. 
# Ex: if the token is RM170.00, now in labelling the actual value of amount is just 170.00, but since this value is sub-span of this whole token, 
# the encoding cannnot be done eeasily, therefore we to take care of such cases we take the whole token rather than just the sub-span.

print('number of Date labels', sum([max(x) for x in data.dateTarget]))
print('number of Name labels', sum([max(x) for x in data.nameTarget]))
print('number of Address labels', sum([max(x) for x in data.addressTarget]))
print('number of Amount labels', sum([max(x) for x in data.amountTarget]))

Before we proceed lets invistigate if everything is okay in terms of data frame creation or not. We will do this by extracting the token from annotations encoding for each label and comparing with the actuals.

In [ ]:
## get the tokens set which form part of each label.

temp = [np.array(token)[np.array(mask == 1)] for (token, mask) in zip(data.sentTokens, data.dateTarget) ]
data['dateTokens'] = temp

temp = [np.array(token)[np.array(mask == 1)] for (token, mask) in zip(data.sentTokens, data.nameTarget) ]
data['nameTokens'] = temp

temp = [np.array(token)[np.array(mask == 1)] for (token, mask) in zip(data.sentTokens, data.addressTarget) ]
data['addressTokens'] = temp

temp = [np.array(token)[np.array(mask == 1)] for (token, mask) in zip(data.sentTokens, data.amountTarget) ]
data['amountTokens'] = temp


In [ ]:
# lets check the whole dataframe
data

In [ ]:
## Lets read the actual text which form part of each label
# placeholder for annotations text
placeholder = {'Date':['null'], 'Address': ['null'], 'Name':['null'], 'Amount':['null']}

for keys in ['Date', 'Address', 'Name', 'Amount']: 
    # read annotation_start, annotation_end and text from data
    for a,b,c in data[['{}_start'.format(keys), '{}_end'.format(keys), 'text']].itertuples(index=False):
        # extracting the text
        if not np.isnan(a) and  not np.isnan(b):
            placeholder[keys].append(c[int(a):int(b)])
        # if annotation not present then append None
        else:
            placeholder[keys].append(None)

placeholder['dateTextactual'] = placeholder['Date'][1:]
placeholder['addressTextactual'] = placeholder['Address'][1:]
placeholder['nameTextactual'] = placeholder['Name'][1:]
placeholder['amountTextactual'] = placeholder['Amount'][1:]

del placeholder['Address'], placeholder['Amount'], placeholder['Date'], placeholder['Name']

y = pd.DataFrame.from_dict(placeholder)

###-------Lets construct the each annotations from tokens and targetencoding
temp = [' '.join([str(tokens) for tokens in list_])      for list_ in data.dateTokens]
y['dateFromTokens'] = temp

temp = [' '.join([str(tokens) for tokens in list_])      for list_ in data.nameTokens]
y['nameFromTokens'] = temp

temp = [' '.join([str(tokens) for tokens in list_])      for list_ in data.addressTokens]
y['addressFromTokens'] = temp

temp = [' '.join([str(tokens) for tokens in list_])      for list_ in data.amountTokens]
y['amountFromTokens'] = temp

In [ ]:
# just for demo..purpose
y

Lets start building the model now. We will start with feature extraction. For this we will use the spacy to give features values for each token. 

> 
*   Feature1: index of token in the invoice
*   Feature2: Is token alphanumeric or not
*   Feature3: Whether Token is like Num or not ( its different from is-alpha Ex: 01-11189 will have like_Num =  True and is_alpha = False, but the token 21:15 will have like_Num = False and is_alpha = False, as the other is likely representing time and is not purely a Number)
*   Feature4: length of token
Lets have a look what kind of values we get one invoice

In [ ]:
##lets explore what kind of values we get, and what it means by these features

# function to take the list of tokens for each invoice and return the features for each token

def invoiceFeatures(sent,polygons): 
    idx = 0
    temp1 = []
    temp2 = []
    for word, box in zip(sent, polygons):
        box[0]
        # as some token dont have x or y coordinate if it start from 0, therefore need to check for that
        indices = [ checkMissingCoordinate(box[0]), checkMissingCoordinate(box[1]), checkMissingCoordinate(box[2]), checkMissingCoordinate(box[3])]
        temp1.append(indices)
        feat = [idx, word.isalpha(), word.isalnum(), len(word)]
        temp2.append(feat)
        idx +=1
    temp1, temp2  = np.array(temp1), np.array(temp2)
    return np.concatenate((temp1.reshape(temp1.shape[0],8) , temp2), axis = 1)

# function to iterate over the all the invoice, this will call sent2features internally
def prep_features(sents,polygonsList):
    sent_features = []
    
    for sent,polygons in zip(sents, polygonsList):
        temp = invoiceFeatures(sent, polygons)
        temp = np.transpose(temp)
        temp = tf.keras.preprocessing.sequence.pad_sequences(temp, padding='post', maxlen= max_len, value = -1, dtype= 'float32')
        temp = np.transpose(temp)
        # print(temp)
        sent_features.append(temp)
    return np.array(sent_features)

def checkMissingCoordinate(coordinatePair):
    temp = []

    if 'x' in coordinatePair.keys():
        temp.append(coordinatePair['x'])
    else:
        temp.append(0.0)
    
    if 'y' in coordinatePair.keys():
        temp.append(coordinatePair['y'])
    else: 
        temp.append(0.0)
    
    return temp

  
# this is just some tokenizing technique using keras and create the library of tokens...
# lets build custom tokenizer adapted for this corpus

def tokenize(lang, num_words = None):
    # lang = list of sentences in a language

    # lets select the default config and basic Tokenizer from keras
    
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', num_words= num_words)

    # this step is necessary to allow tokenzier build its words id for the corpus 
    # from the lang = list of sentences
    lang_tokenizer.fit_on_texts(lang)

    ## tf.keras.preprocessing.text.Tokenizer.texts_to_sequences converts string (w1, w2, w3, ......, wn) 
    ## to a list of correspoding integer ids of words (id_w1, id_w2, id_w3, ...., id_wn)
    tensor = lang_tokenizer.texts_to_sequences(lang) 

    ## tf.keras.preprocessing.sequence.pad_sequences takes argument a list of integer id sequences 
    ## and pads the sequences to match the longest sequences in the given input
    # this will make all sentences of same length
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', value = 0)

    return tensor, lang_tokenizer

In [ ]:

# getting max length of sequence of tokens for invoices
# we need this variable for padding

max_len = max([len(x) for x in data.sentTokens])
print("max length is {}".format(max_len))

# we will be dropping the data where any label is missing, this will be done in training and validation
type_data = ['TRAIN', 'VALIDATION', 'TEST']
d_train = data[data.dataType == 'TRAIN']
d_valid =  data[data.dataType == 'VALIDATION']

d_train = d_train.dropna(axis = 0, subset= ['Date_start','Name_start','Address_start','Amount_start','Date_end','Address_end','Name_end','Amount_end'])
d_valid = d_valid.dropna(axis = 0, subset= ['Date_start','Name_start','Address_start','Amount_start','Date_end','Address_end','Name_end','Amount_end'])

X_train_feat = prep_features(d_train.sentTokens, d_train.boundingPoly)
X_valid_feat = prep_features(d_valid.sentTokens, d_valid.boundingPoly)
X_test_feat = prep_features(data[data.dataType == 'TEST'].sentTokens, data[data.dataType == 'TEST'].boundingPoly)


In [ ]:
# getting token ids for each token and padding each invoice
# Further we split the data (train, validation and test)
# splitting the data

X_train, tokenizer_ = tokenize(d_train.sentTokens)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len, padding='post', value = 0)

X_valid = tokenizer_.texts_to_sequences(d_valid.sentTokens)
X_valid = tf.keras.preprocessing.sequence.pad_sequences(X_valid, maxlen=max_len, padding='post', value = 0)

X_test = tokenizer_.texts_to_sequences(data[data.dataType == 'TEST'].sentTokens)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len, padding='post', value = 0)

In [ ]:
# lets have a look at the shapes of data

print("X_TRAIN",X_train.shape)
print("Train_features",X_train_feat.shape)
print("X_VALID",X_valid.shape)
print("Valid_features",X_valid_feat.shape)

In [ ]:
# this is required as to tell the input shape to model.
shape_ = X_train_feat.shape

In [ ]:
def build_model():

  # we will create two inputs one which has sequence of just tokens and
  # other which has feature of each token (so array of features of each token)
  x1 = Input(shape=(max_len,))
  x2 = Input(shape = (max_len,X_train_feat.shape[2]))
#----------------------------------------TOKEN Embedding -----------------------------------

  mask1 = layers.Masking(mask_value=0)(x1)
  embed = layers.Embedding(input_dim= len(tokenizer_.word_index), output_dim=128, input_length=max_len)(mask1)
  norm1 = layers.BatchNormalization(axis=1)(embed)

#---------------------------------------------FEATURE-----------------------------

  mask2 = layers.Masking(mask_value=-1)(x2)
#----------------------------------------------Merging both---NOW first we pass through memory(LSTM) and Dense(for more complex fetaure)......

  merge = layers.concatenate([norm1,mask2])
  drop1 = layers.Dropout(0.2)(merge)
  lstm1 = layers.LSTM(units= 300, return_sequences= True)(drop1)
  denseLinear = layers.Dense(units= 300, activation = 'relu')(lstm1)
# ------------------------ and now attention starts ( attention acts on memory (LSTM)-----------------------
  attention = layers.Dense(1, activation='tanh')(lstm1)
  attention = layers.Flatten()(attention)
  attention = layers.Activation('softmax')(attention)
  attention = layers.RepeatVector(300)(attention)
  attention = layers.Permute([2, 1])(attention)
# -------------------------------------------------Attention tells LSTM what to remember and what to forget---------------
  sent_representation = layers.Multiply()([lstm1, attention])

  dense = layers.Dense(512, activation = 'relu')(sent_representation)
  drop1 = layers.Dropout(0.3)(dense)
  dense2 = layers.Dense(128)(drop1)

# ------------------------------------------------------------------------------------

#   # KEEP THIS INTACT
  predictions = layers.Dense(5, activation='softmax')(dense2)

  model = Model(inputs = ([x1,x2]), outputs = predictions)

  print(model.summary())

  return model


In [ ]:
# since now we ahve all annotations together we create target encoding for same

def createTargetForMultilabel():
   # with this now 0: No label, 1: name, 2: address, 3: date, 4: amount
  d_train['tags'] = d_train.nameTarget + d_train.addressTarget*2 + d_train.dateTarget*3 + d_train.amountTarget*4 
  a = d_train.tags.tolist()
  y_train = tf.keras.preprocessing.sequence.pad_sequences(a, padding='post', maxlen= max_len,dtype='int32', value = 0)

  d_valid['tags'] = d_valid.nameTarget + d_valid.addressTarget*2 + d_valid.dateTarget*3 +  d_valid.amountTarget*4
  a = d_valid.tags.tolist()
  y_valid = tf.keras.preprocessing.sequence.pad_sequences(a, padding='post', maxlen= max_len,dtype='int32', value = 0)

# one hot encode target
  y_train = [tf.keras.utils.to_categorical(i, num_classes=5) for i in y_train]
  y_valid = [tf.keras.utils.to_categorical(i, num_classes=5) for i in y_valid]

  y_train = np.array(y_train)
  y_valid = np.array(y_valid)
  return y_train, y_valid


In [ ]:
# function to plot the loss, precision and recall, 
# caution code block has lines to save the files to drive.

def display_history(history,name ="None"):

    import seaborn as sns
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np                                        
    # notebook and use it by doing slight modification
    # """Summarize history for accuracy and loss.

    sns.set_palette("pastel")
    sns.set(style="darkgrid")
    path = '/content/drive/MyDrive/..../'

    fig, ax = plt.subplots(figsize=(10,6),sharey='row')
    title = "Loss_for_{}".format(name)
    sns.lineplot(data=history.history['loss'], ax = ax)
    sns.lineplot(data=history.history['val_loss'], ax = ax)
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.title(title)
    fig.savefig(path + title + '.png')

    fig, ax = plt.subplots(figsize=(10,6),sharey='row')
    title = "Precision_for_{}".format(name)
    sns.lineplot(data=history.history['precision'], ax = ax)
    sns.lineplot(data=history.history['val_precision'], ax = ax)
    ax.set_xlabel('epoch')
    ax.set_ylabel('precision')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.title(title)
    fig.savefig( path + title + '.png')

    fig, ax = plt.subplots(figsize=(10,6),sharey='row')
    title = "Recall_for_{}".format(name)
    sns.lineplot(data=history.history['recall'], ax = ax)
    sns.lineplot(data=history.history['val_recall'], ax = ax)
    ax.set_xlabel('epoch')
    ax.set_ylabel('recall')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.title(title)
    fig.savefig( path + title + '.png')

In [ ]:
# 0: None, 1: name, 2: address, 3: date, 4: amount
def weighted_loss(class_weights):
    weights = list(class_weights.values())
    def loss(y_true, y_pred):

        y_pred = be.clip(y_pred, be.epsilon(), 1 - be.epsilon())
        
        loss = y_true * be.log(y_pred) * weights
        loss = -be.sum(loss, axis= -1)
        return loss
    
    return loss

In [ ]:
# this will take the tag = 'date'/'name'...make the target encoding as per that and load and compile the model
def LoadandCompile(tag):
  be.clear_session()

  y_train, y_valid =  createTargetForMultilabel()
  class_weights = {0:0.05, 1:100, 2:20, 3: 200, 4: 180}
  loss = weighted_loss(class_weights = class_weights)

  optimizer=RectifiedAdam(learning_rate = 0.001, warmup_proportion = 0.3, beta_1 = 0.9,weight_decay= 0.05,   
                                            total_steps= 2500, min_lr= 0.0001)
  model = build_model()
  model.compile(optimizer=optimizer,
                loss= loss,
                metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  
  return model, y_train, y_valid  

In [ ]:
# lets run the model for each annotation

be.clear_session()

model,y_train, y_valid = LoadandCompile('combined')

print("Starting training {} on with model presented above".format('combined'))
history = model.fit(x=[X_train,X_train_feat], y=y_train,
                  validation_data=([X_valid,X_valid_feat], y_valid),
                  epochs=1200,batch_size=32) 

display_history(history, name = 'combined')

In [ ]:
model.save('/content/drive/MyDrive/ING/{}_combined.h5'.format('attention'))

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/ING/Attention/final/attention_combined_v3.h5', compile = False)

In [ ]:
model.summary()

In [ ]:
# lets do the prediction on test now and see the metrics
a = model.predict([X_test,X_test_feat])

b = a.tolist()
df = data[data.dataType == 'TEST'].reset_index()
df['probs'] = b
def f(x,y,temp):
  return(y[:len(x),temp])

df['predictions'] = df.probs.apply(lambda x: (np.array(x) == np.array(x).max(axis =1)[:, None]).astype(int))
df['predictedname'] = df.apply(lambda x: f(x.sentTokens,x.predictions,1), axis = 1)
df['predictedaddress'] = df.apply(lambda x: f(x.sentTokens,x.predictions,2), axis = 1)
df['predicteddate'] = df.apply(lambda x: f(x.sentTokens,x.predictions,3), axis = 1)
df['predictedamount'] = df.apply(lambda x: f(x.sentTokens,x.predictions,4), axis = 1)


#metric evlauation function
def onehotinvoicesmetric(tag):
  TP = 0
  for a,b in zip(df['{}Target'.format(tag)],df['predicted{}'.format(tag)]):
    count = (a == b)
    if type(count) == type(True):
      TP += 1
    elif sum(count) == len(a):
      TP += 1
  return (TP/len(df))*100


In [ ]:
print('DATE',onehotinvoicesmetric('date'))
print('AMOUNT', onehotinvoicesmetric('amount'))
print('NAME', onehotinvoicesmetric('name'))
print('ADDRESS',onehotinvoicesmetric('address'))